In [1]:
import pandas as pd
import sqlalchemy as sql
import main as m
import betSim as bs
import mysql.connector as mysql

In [8]:
df = m.getTable('LCK_Spring_2022')

c:\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [9]:
df

,gameID,matchID,tournamentID,matchName,tournament_name,region,Num_in_Match,Blue_Team_Name,Red_Team_Name,Date,...,First_tower_time,First_dragon_team,First_dragon_time,First_rift_herald_team,First_rift_herald_time,First_baron_team,First_baron_time,Game_time,Blue_players,Red_players
0,11322,11322,1,DRX vs LSB,LCK_Spring_2022,LCK,1,DRX,Liiv SANDBOX,2022-01-12,...,673,blue,516,blue,516,red,1590,2195,"kingen,Pyosik,Zeka,Deft,BeryL,","Dove,Croco,Clozer,Envyy,Kael,"
1,21322,11322,1,DRX vs LSB,LCK_Spring_2022,LCK,2,DRX,Liiv SANDBOX,2022-01-12,...,1131,red,192,blue,509,red,1562,2069,"kingen,Pyosik,Zeka,Deft,BeryL,","Dove,Croco,Clozer,Envyy,Kael,"
2,31322,21322,1,T1 vs KDF,LCK_Spring_2022,LCK,1,T1,Kwangdong Freecs,2022-01-12,...,922,blue,366,blue,526,blue,1985,2232,"Zeus,Oner,Faker,Gumayusi,Keria,","Kiin,Ellim,FATE,Teddy,Hoit,"
3,41322,21322,1,T1 vs KDF,LCK_Spring_2022,LCK,2,Kwangdong Freecs,T1,2022-01-12,...,538,red,379,blue,506,red,1279,1505,"Kiin,Ellim,FATE,Teddy,Hoit,","Zeus,Oner,Faker,Gumayusi,Keria,"
4,51322,31322,1,HLE vs NS,LCK_Spring_2022,LCK,1,Nongshim RedForce,Hanwha Life eSports,2022-01-13,...,861,blue,466,blue,512,blue,1674,2152,"Canna,Dread,Bdd,Ghost,Effort,","DuDu,OnFleek,Karis,SamD,Vsta,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,2081322,881322,1,KDF vs LSB,LCK_Spring_2022,LCK,1,Kwangdong Freecs,Liiv SANDBOX,2022-03-20,...,1012,red,318,red,530,blue,1492,1918,"Kiin,Ellim,FATE,Teddy,Hoit,","Dove,Croco,Clozer,Ice,Kael,"
208,2091322,881322,1,KDF vs LSB,LCK_Spring_2022,LCK,2,Liiv SANDBOX,Kwangdong Freecs,2022-03-20,...,932,blue,391,blue,611,red,1734,2094,"Dove,Croco,Clozer,Ice,Kael,","Kiin,Ellim,FATE,Teddy,Hoit,"
209,2101322,891322,1,T1 vs DRX,LCK_Spring_2022,LCK,1,T1,DRX,2022-03-20,...,974,red,360,red,556,none,0,1764,"Zeus,Oner,Faker,Gumayusi,Keria,","kingen,Pyosik,Zeka,Deft,BeryL,"
210,2111322,891322,1,T1 vs DRX,LCK_Spring_2022,LCK,2,DRX,T1,2022-03-20,...,857,red,60,blue,540,red,1472,1831,"kingen,Pyosik,Zeka,Deft,BeryL,","Zeus,Oner,Faker,Gumayusi,Keria,"


In [ ]:
# very basic simulation of a beting strategy
units =100.0
for i in range(len(df)):
    if (units < 1):
        print(f'no more units left after {str(i)} games')
        break
    outcome = bs.towerBet(i, df, 'under', 11.5)
    if(outcome == 1):
        units += 0.83
    else:
        units -= 1

print(f'units: {str(units)}')

In [ ]:

db = mysql.connect(host='140.238.205.186', user='loki', passwd='Thethethe3!', database='lolz')
mycursor = db.cursor()

In [ ]:
mycursor.execute('SHOW DATABASES')
print(mycursor.fetchall())

In [4]:
engine = sql.create_engine